**This Dataset contains**

Label:

    1 represents fake

    0 represents real

**Importing necessary and suitable Libraries**

In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [84]:
import re #regular expression
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [85]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [86]:
#printing stopwords

print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

**Data Preprocessing**

In [87]:
#loading dataset

data=pd.read_csv('train.csv', engine='python', on_bad_lines='skip')

In [88]:
data.shape

(20822, 5)

In [89]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [90]:
#counting number of missing values in the dataset

data.isnull().sum()

,0
id,0
title,565
author,1975
text,59
label,22


In [91]:
#replacing the missing values with the null string

data=data.fillna('')

In [92]:
# for prediction creating new column named as content by merging title and author strings

data['content']=data['author']+' '+data['title']

In [93]:
data['content']

,content
0,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,Consortiumnews.com Why the Truth Might Get You...
3,Jessica Purkiss 15 Civilians Killed In Single ...
4,Howard Portnoy Iranian woman jailed for fictio...
...,...
20817,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20818,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20819,Michael J. de la Merced and Rachel Abrams Macy...
20820,"Alex Ansary NATO, Russia To Hold Parallel Exer..."


In [94]:
# seperating the label column with main dataset

X=data.drop(columns='label', axis=1)
Y=data['label']

In [95]:
X

,id,title,author,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...
20817,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20818,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20819,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,Michael J. de la Merced and Rachel Abrams Macy...
20820,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...","Alex Ansary NATO, Russia To Hold Parallel Exer..."


In [96]:
Y

,label
0,1
1,0
2,1
3,1
4,1
...,...
20817,0
20818,0
20819,0
20820,1


**Stemming**

In [97]:
#using PorterStemmer

port_stem=PorterStemmer()

In [98]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-z]',' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [99]:
data['content']=data['content'].apply(stemming)

In [100]:
X=data['content'].values

In [101]:
X

array(['darrel lucu hous dem aid even see comey letter jason chaffetz tweet',
       'daniel j flynn flynn hillari clinton big woman campu breitbart',
       'consortiumnew com truth might get fire', ...,
       'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time',
       'alex ansari nato russia hold parallel exercis balkan',
       'david swanson keep f aliv'], dtype=object)

In [102]:
Y

,label
0,1
1,0
2,1
3,1
4,1
...,...
20817,0
20818,0
20819,0
20820,1


In [103]:
print(X.shape)
print(Y.shape)

(20822,)
(20822,)


In [104]:
# converting the textual data to numerical data

vectorizer = TfidfVectorizer()
vectorizer.fit(X)

TfidfVectorizer()

In [105]:
X=vectorizer.transform(X)

In [106]:
X

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 210723 stored elements and shape (20822, 17149)>

**Splitting the dataset into train and test data**

In [107]:
Y = pd.to_numeric(Y, errors='coerce')
valid_indices = Y.dropna().index
X = X[valid_indices]
Y = Y.loc[valid_indices].astype(int)
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

**Training the Model: Logistic Regression**

In [108]:
model=LogisticRegression()

In [109]:
model.fit(X_train, Y_train)

LogisticRegression()

**Evaluation: accuracy score**

In [110]:
# accuracy score on the training data

X_train_pred=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_pred,Y_train)

In [111]:
training_data_accuracy

0.9868974636374565

In [112]:
# accuracy score on the testing data

X_test_pred=model.predict(X_test)
testing_data_accuracy=accuracy_score(X_test_pred,Y_test)

In [113]:
testing_data_accuracy

0.9783653846153846

**Making Predictive system**

In [114]:
X_new=X_test[0]

X_pre=model.predict(X_new)
print(X_pre)

if (X_pre[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[0]
The news is Real


In [115]:
print(Y_test.iloc[0])

0
